In [2]:
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from PIL import Image
from AB3DMOT_libs.kitti_utils import read_label, compute_box_3d, draw_projected_box3d, Calibration, Object3d

# Visualize detection result
Load detection results from "data/KITTI/result_sha/seq" and images from "data_root/image_02/seq". Save visualization results in "./results/result_sha/det_image_vis/seq" 

In [3]:
def visdet(result_sha, data_root, seq):
    width = 1242
    height = 374
    def show_image_with_boxes(img, objects_res, object_gt, calib, save_path, height_threshold=0):
        img2 = np.copy(img) 
        # for each object, compute the bouding box in 2D image. Use the same color for objects of same obj.id. 
        for obj in objects_res:
            box3d_pts_2d, _ = compute_box_3d(obj, calib.P) # calib.P is the projection matrix from camera coord to image coord
            color_tmp = tuple([255,0,0])
            img2 = draw_projected_box3d(img2, box3d_pts_2d, color=color_tmp)
            text = 'ID: %d' % obj.id
            if box3d_pts_2d is not None:
                img2 = cv2.putText(img2, text, (int(box3d_pts_2d[4, 0]), int(box3d_pts_2d[4, 1]) - 8), cv2.FONT_HERSHEY_TRIPLEX, 0.5, color=color_tmp) 

        img = Image.fromarray(img2)
        img = img.resize((width, height))
        img.save(save_path)
    image_dir = os.path.join(data_root, 'image_02/%s' % seq)
    calib_file = os.path.join('data/KITTI/resources/training/', 'calib/%s.txt' % seq) 

    save_3d_bbox_dir = os.path.join('./results',result_sha,'det_image_vis/%s' % seq)
    print(save_3d_bbox_dir)
    if not os.path.exists(save_3d_bbox_dir):
        os.makedirs(save_3d_bbox_dir)

    seq_file = os.path.join('data/KITTI', result_sha, "%s.txt"%seq)
    seq_dets = np.loadtxt(seq_file, delimiter=',')
    if len(seq_dets.shape) == 1: seq_dets = np.expand_dims(seq_dets, axis=0) 	
    if seq_dets.shape[1] == 0:
        return
    calib_tmp = Calibration(calib_file)
    # loop over frame
    min_frame, max_frame = int(seq_dets[:, 0].min()), int(seq_dets[:, 0].max())
    print(min_frame, max_frame)
    for frame in range(min_frame, max_frame + 1):
        try:
            img_tmp = np.array(Image.open(os.path.join(image_dir,"%06d.png"%frame)))
        except Exception as e:
            print(e)
            break
        # logging
        print_str = 'processing %s:  %d/%d   \r' % (seq,  frame, max_frame)
        print(print_str)

        dets = seq_dets[seq_dets[:,0] == frame]            # h, w, l, x, y, z, theta in camera coordinate follwing KITTI convention
        objects = []
        for i, det in enumerate(dets):
            str_to_srite = '%s -1 -1 %f %f %f %f %f %f %f %f %f %f %f %f %f %d\n' % ("Car", det[14],
                    det[2], det[3], det[4], det[5], 
                    det[7], det[8], det[9], det[10], det[11], det[12], det[13], det[6], 1)
            objects.append(Object3d(str_to_srite))
        save_path_tmp = os.path.join(save_3d_bbox_dir, '%06d.jpg' % frame)
        show_image_with_boxes(img_tmp, objects, [], calib_tmp, save_path_tmp)

## Example

I put 21 images in ./data/KITTI/resources/testing/image_02/seq

In [8]:
result_sha = "pointrcnn_thresh_world_eval_1/Car"
data_root = '/team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/'
calib = 'data/KITTI/resources/training/'
seqs = ["%04d"%i for i in range(21) if i!=11]
for seq in seqs:
    visdet(result_sha, data_root, seq)

./results/pointrcnn_thresh_world_eval_1/Car/det_image_vis/0000
0 153
Trying frame 0 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000000.png
processing 0000:  0/153   
Trying frame 1 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000001.png
processing 0000:  1/153   
Trying frame 2 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000002.png
processing 0000:  2/153   
Trying frame 3 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000003.png
processing 0000:  3/153   
Trying frame 4 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000004.png
processing 0000:  4/153   
Trying frame 5 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000005.png
processing 0000:  5/153   
Trying frame 6 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000006.png
processing 0000:  6/153   


processing 0000:  60/153   
Trying frame 61 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000061.png
processing 0000:  61/153   
Trying frame 62 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000062.png
processing 0000:  62/153   
Trying frame 63 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000063.png
processing 0000:  63/153   
Trying frame 64 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000064.png
processing 0000:  64/153   
Trying frame 65 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000065.png
processing 0000:  65/153   
Trying frame 66 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000066.png
processing 0000:  66/153   
Trying frame 67 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000067.png
processing 0000:  67/153   
Trying frame 68 /team1/code

processing 0000:  121/153   
Trying frame 122 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000122.png
processing 0000:  122/153   
Trying frame 123 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000123.png
processing 0000:  123/153   
Trying frame 124 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000124.png
processing 0000:  124/153   
Trying frame 125 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000125.png
processing 0000:  125/153   
Trying frame 126 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000126.png
processing 0000:  126/153   
Trying frame 127 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000127.png
processing 0000:  127/153   
Trying frame 128 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0000/000128.png
processing 0000:  128/153   
Trying frame

processing 0001:  27/442   
Trying frame 28 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000028.png
processing 0001:  28/442   
Trying frame 29 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000029.png
processing 0001:  29/442   
Trying frame 30 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000030.png
processing 0001:  30/442   
Trying frame 31 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000031.png
processing 0001:  31/442   
Trying frame 32 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000032.png
processing 0001:  32/442   
Trying frame 33 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000033.png
processing 0001:  33/442   
Trying frame 34 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000034.png
processing 0001:  34/442   
Trying frame 35 /team1/code

Trying frame 89 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000089.png
processing 0001:  89/442   
Trying frame 90 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000090.png
processing 0001:  90/442   
Trying frame 91 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000091.png
processing 0001:  91/442   
Trying frame 92 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000092.png
processing 0001:  92/442   
Trying frame 93 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000093.png
processing 0001:  93/442   
Trying frame 94 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000094.png
processing 0001:  94/442   
Trying frame 95 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0001/000095.png
processing 0001:  95/442   
Trying frame 96 /team1/codes/3dObjDet/OpenPCDet_ravi/da

...

processing 0020:  565/836   
Trying frame 566 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000566.png
processing 0020:  566/836   
Trying frame 567 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000567.png
processing 0020:  567/836   
Trying frame 568 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000568.png
processing 0020:  568/836   
Trying frame 569 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000569.png
processing 0020:  569/836   
Trying frame 570 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000570.png
processing 0020:  570/836   
Trying frame 571 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000571.png
processing 0020:  571/836   
Trying frame 572 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000572.png
processing 0020:  572/836   
Trying frame

processing 0020:  625/836   
Trying frame 626 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000626.png
processing 0020:  626/836   
Trying frame 627 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000627.png
processing 0020:  627/836   
Trying frame 628 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000628.png
processing 0020:  628/836   
Trying frame 629 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000629.png
processing 0020:  629/836   
Trying frame 630 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000630.png
processing 0020:  630/836   
Trying frame 631 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000631.png
processing 0020:  631/836   
Trying frame 632 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000632.png
processing 0020:  632/836   
Trying frame

processing 0020:  686/836   
Trying frame 687 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000687.png
processing 0020:  687/836   
Trying frame 688 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000688.png
processing 0020:  688/836   
Trying frame 689 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000689.png
processing 0020:  689/836   
Trying frame 690 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000690.png
processing 0020:  690/836   
Trying frame 691 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000691.png
processing 0020:  691/836   
Trying frame 692 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000692.png
processing 0020:  692/836   
Trying frame 693 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000693.png
processing 0020:  693/836   
Trying frame

processing 0020:  746/836   
Trying frame 747 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000747.png
processing 0020:  747/836   
Trying frame 748 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000748.png
processing 0020:  748/836   
Trying frame 749 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000749.png
processing 0020:  749/836   
Trying frame 750 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000750.png
processing 0020:  750/836   
Trying frame 751 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000751.png
processing 0020:  751/836   
Trying frame 752 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000752.png
processing 0020:  752/836   
Trying frame 753 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000753.png
processing 0020:  753/836   
Trying frame

processing 0020:  807/836   
Trying frame 808 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000808.png
processing 0020:  808/836   
Trying frame 809 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000809.png
processing 0020:  809/836   
Trying frame 810 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000810.png
processing 0020:  810/836   
Trying frame 811 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000811.png
processing 0020:  811/836   
Trying frame 812 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000812.png
processing 0020:  812/836   
Trying frame 813 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000813.png
processing 0020:  813/836   
Trying frame 814 /team1/codes/3dObjDet/OpenPCDet_ravi/data/kitti-odometry/training/image_02/0020/000814.png
processing 0020:  814/836   
Trying frame

## Create video


In [6]:
seqs = ["%04d"%i for i in range(21) if i!=11]
seqs = ["0012"]
model_d = 'pointrcnn_thresh_world_eval_1'
for seq in seqs:
    print(seq)
#     result_dir = "./results/videos/%s_Car/trk_image_vis/%s/"%(model_d,seq)
    result_dir = "./results/%s/Car/det_image_vis/%s/"%(model_d,seq)
    num_imgs = len(os.listdir(result_dir))
    imgs = []
    for img_idx in range(num_imgs):
        img_path = os.path.join(result_dir, '%06d.jpg'%img_idx)
        imgs.append(cv2.imread(img_path))
    video = cv2.VideoWriter("./videos/%s/%s.mp4"%(model_d,seq), cv2.VideoWriter_fourcc(*'XVID'), 10, (1242,374))
    for image in imgs:
       # plt.imshow(image)
        video.write(image)
    video.release()

0012
